# Shear Likelihood 1

### Preparing necessary libraries

In [1]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from numpy import random
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units

### Creating mock data

In [2]:
np.random.seed(11)

cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

cluster_m     = 1.e15
logm = np.log(cluster_m)/np.log(10)
cluster_z     = 0.4
concentration = 4
ngals         = 10000
Delta         = 200
cluster_ra    = 20.0
cluster_dec   = 90.0

ideal_data   = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 0.8, zsrc_min = cluster_z + 0.1, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
ideal_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo,'chang13', zsrc_min = cluster_z + 0.1, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
noisy_data_z = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1, shapenoise=0.05, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)

gc_ideal   = clmm.GalaxyCluster("CL_ideal", cluster_ra, cluster_dec, cluster_z, ideal_data)
gc_ideal_z = clmm.GalaxyCluster("CL_ideal_z", cluster_ra, cluster_dec, cluster_z, ideal_data_z)
gc_noisy_z = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, noisy_data_z)

# For some reason galaxy catalogs access information very deifferently from galaxy clusters

### Calculating observables

In [3]:
obs_ideal   = gc_ideal.compute_tangential_and_cross_components(geometry="flat")
obs_ideal_z = gc_ideal_z.compute_tangential_and_cross_components(geometry="flat")
obs_noisy_z = gc_noisy_z.compute_tangential_and_cross_components(geometry="flat")

# Convert from theta to radius
radius = [convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=cluster_z, cosmo=cosmo) for gc in [gc_ideal, gc_ideal_z, gc_noisy_z]]

### Binning the data

In [4]:
bin_edges = da.make_bins(0.7, 4, 15, method='evenlog10width')

profile_ideal   = gc_ideal.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_ideal_z = gc_ideal_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)
profile_noisy_z = gc_noisy_z.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

profiles = [profile_ideal, profile_ideal_z, profile_noisy_z]

/home/caiooliveira/.local/lib/python3.10/site-packages/clmm-1.1.10-py3.10.egg/clmm/utils.py:176: RuntimeWarning: invalid value encountered in sqrt


In [5]:
print(sum(r <= 4. and r >= 0.7 for r in radius[0]))
print(sum([len(profile_ideal[i]['gal_id']) for i in range(0,15)]))

7597
7597


In [41]:
# Probability of measuring gt_obs given a certain r, z, cosmo, etc.
def shear_probability(r, gal_z, cluster_z, gt_obs, sigma, cosmo, m=1.e15, concentration=4, delta=200, model='nfw'):

    gt_th = clmm.compute_reduced_tangential_shear(r, m, concentration, cluster_z, gal_z, cosmo, delta_mdef=200, halo_profile_model=model)

    return np.exp(-np.power(np.subtract(gt_obs, gt_th), 2)/2/sigma**2)/np.sqrt(2*np.pi)/sigma

In [43]:
def shear_likelihood(radius, gc_data, cosmo, sigma=1, m=1.e15, concentration=4, delta=200, model='nfw'):

    n_bins = len(gc_data.profile)
    cluster_z = gc_data.z
    likelihood = 1

    for i in range(n_bins):

        gal_list = gc_data.profile['gal_id'][i]
        z_list = gc_data.galcat['z'][gal_list]
        r_list = [radius[gal_id] for gal_id in gal_list]
        shear_obs = gc_data.profile['gt'][i]

        shear_prob_bin = np.mean(shear_probability(r_list, z_list, cluster_z, shear_obs, sigma, cosmo, m, concentration, delta, model))
        likelihood *= shear_prob_bin

    return likelihood

In [46]:
shear_likelihood(radius[0], gc_ideal, cosmo)

1.0258993807047827e-06